In [12]:
import sys
import os
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [2]:
sys.path.append("/home/inoue/work/dataset")

# RecSys Challenge 2015のデータセット

In [3]:
train_df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-clicks.dat", header=None)
test_df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-test.dat", header=None)

/tmp/ipykernel_52154/2692782158.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-clicks.dat", header=None)
/tmp/ipykernel_52154/2692782158.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("/home/inoue/work/dataset/RC15/yoochoose-test.dat", header=None)


In [4]:
train_df.columns = ["sessionId", "timestamp", "itemId", "categoryId"]
test_df.columns = ["sessionId", "timestamp", "itemId", "categoryId"]

In [5]:
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"], format="%Y-%m-%dT%H:%M:%S.%fZ")
test_df["timestamp"] = pd.to_datetime(test_df["timestamp"], format="%Y-%m-%dT%H:%M:%S.%fZ")

In [6]:
train_item_size = train_df.groupby("itemId").size()
train_df = train_df[train_df.itemId.isin(train_item_size[train_item_size > 4].index)]

In [7]:
# sessionの長さが1のものを消去
train_session_len = train_df.groupby("sessionId")["itemId"].apply(len)
train_session_len

sessionId
1           4
2           6
3           3
4           2
6           2
           ..
11562156    2
11562157    2
11562158    3
11562159    1
11562161    1
Name: itemId, Length: 9244493, dtype: int64

In [8]:
train_df = train_df[train_df["sessionId"].isin(train_session_len[train_session_len != 1].index)]

In [9]:
train_df = train_df.sort_values("timestamp")
train_df

,sessionId,timestamp,itemId,categoryId
1031562,351646,2014-04-01 03:00:00.124,214717005,0
1163646,389654,2014-04-01 03:00:00.567,214826705,0
885068,263073,2014-04-01 03:00:10.087,214716982,0
620662,210798,2014-04-01 03:00:13.070,214581827,0
1217521,375257,2014-04-01 03:00:13.768,214644307,0
...,...,...,...,...
32350125,11528554,2014-09-30 02:59:43.577,214572433,S
32653390,11422848,2014-09-30 02:59:47.397,214859908,S
32432376,11474968,2014-09-30 02:59:49.546,214685049,3
32432377,11474968,2014-09-30 02:59:53.315,214685049,3


In [10]:
test_item_size = test_df.groupby("itemId").size()
test_df = test_df[test_df.itemId.isin(test_item_size[test_item_size > 4].index)]

In [11]:
test_session_len = test_df.groupby("sessionId").size()
test_session_len

sessionId
5           3
10          2
15          3
20          4
25          6
           ..
11562140    1
11562145    4
11562150    3
11562155    2
11562160    3
Length: 2307777, dtype: int64

In [12]:
test_df = test_df[test_df.sessionId.isin(test_session_len[test_session_len > 1].index)]

In [79]:
df = pd.concat([train_df, test_df], axis=0)
df

,sessionId,timestamp,itemId,categoryId
1031562,351646,2014-04-01 03:00:00.124,214717005,0
1163646,389654,2014-04-01 03:00:00.567,214826705,0
885068,263073,2014-04-01 03:00:10.087,214716982,0
620662,210798,2014-04-01 03:00:13.070,214581827,0
1217521,375257,2014-04-01 03:00:13.768,214644307,0
...,...,...,...,...
8251786,11299820,2014-09-25 08:17:19.053,214853094,3
8251787,11299815,2014-09-26 09:49:23.602,214854804,S
8251788,11299815,2014-09-26 09:49:41.808,214714715,S
8251789,11299810,2014-09-26 16:00:32.051,214546123,2


In [80]:
from sklearn.preprocessing import LabelEncoder
item_encoder = LabelEncoder()
df["itemId"] = item_encoder.fit_transform(df["itemId"])

In [83]:
df["itemId"] += 1

In [84]:
df

,sessionId,timestamp,itemId,categoryId
1031562,351646,2014-04-01 03:00:00.124,23053,0
1163646,389654,2014-04-01 03:00:00.567,28309,0
885068,263073,2014-04-01 03:00:10.087,23042,0
620662,210798,2014-04-01 03:00:13.070,8519,0
1217521,375257,2014-04-01 03:00:13.768,12927,0
...,...,...,...,...
8251786,11299820,2014-09-25 08:17:19.053,35672,3
8251787,11299815,2014-09-26 09:49:23.602,36212,S
8251788,11299815,2014-09-26 09:49:41.808,22494,S
8251789,11299810,2014-09-26 16:00:32.051,3285,2


In [86]:
train_df["itemId"] = item_encoder.transform(train_df["itemId"]) + 1
test_df["itemId"] = item_encoder.transform(test_df["itemId"]) + 1

In [25]:
train_df.to_pickle("/home/inoue/work/dataset/RC15/derived/train.df")
test_df.to_pickle("/home/inoue/work/dataset/RC15/derived/test.df")

In [13]:
test_df = pd.read_pickle("/home/inoue/work/dataset/RC15/derived/test.df")
test_df

,sessionId,timestamp,itemId,categoryId
0,5,2014-04-07 17:13:46.713,861,0
1,5,2014-04-07 17:20:56.973,861,0
2,5,2014-04-07 17:21:19.602,861,0
3,10,2014-04-04 07:44:14.590,27702,0
4,10,2014-04-04 07:45:20.245,28341,0
...,...,...,...,...
8251786,11299820,2014-09-25 08:17:19.053,35672,3
8251787,11299815,2014-09-26 09:49:23.602,36212,S
8251788,11299815,2014-09-26 09:49:41.808,22494,S
8251789,11299810,2014-09-26 16:00:32.051,3285,2


In [15]:
train_df = pd.read_pickle("/home/inoue/work/dataset/RC15/derived/train.df")
tra_item = train_df["itemId"].unique()

In [17]:
test_df = test_df[test_df["itemId"].isin(tra_item)]

In [23]:
test_sess_len = test_df.groupby("sessionId").size()
test_df = test_df[test_df["sessionId"].isin(test_sess_len[test_sess_len != 1].index)]
test_df

,sessionId,timestamp,itemId,categoryId
0,5,2014-04-07 17:13:46.713,861,0
1,5,2014-04-07 17:20:56.973,861,0
2,5,2014-04-07 17:21:19.602,861,0
3,10,2014-04-04 07:44:14.590,27702,0
4,10,2014-04-04 07:45:20.245,28341,0
...,...,...,...,...
8251786,11299820,2014-09-25 08:17:19.053,35672,3
8251787,11299815,2014-09-26 09:49:23.602,36212,S
8251788,11299815,2014-09-26 09:49:41.808,22494,S
8251789,11299810,2014-09-26 16:00:32.051,3285,2


In [24]:
test_df.nunique()

sessionId     1990860
timestamp     7901901
itemId          27566
categoryId        275
dtype: int64